Author: Zheng Liu

In this notebook, we focus on
- model saving/loading [(link)](https://www.tensorflow.org/api_docs/python/tf/train/Saver): save and load training progresses
- Tensorboard [(link)](https://www.tensorflow.org/guide/summaries_and_tensorboard): monitor training progress in real time

To use tensorboard, port forwarding setups are needed (similar to ipytho notebook). Once port forwarding is configured, we can run the following command in terminal to start tensorboard:

`tensorboard --logdir DIR_NAME --port=PORT_NUMBER`

In [ ]:
from __future__ import print_function

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf

In [ ]:
# Parameters
num_steps = 201
batch_size = 128
display_step = 100

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 64 # 2nd layer number of neurons
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [ ]:
def neural_net_cnn(x):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    # first layer
    conv1 = tf.layers.conv2d(inputs = x, filters = 16, kernel_size = 2, strides = 1, activation = tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs = conv1, pool_size = 2, strides = 2)
    # second layer
    conv2 = tf.layers.conv2d(inputs = pool1, filters = 16, kernel_size = 2, strides = 1, activation = tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs = conv2, pool_size = 2, strides = 2)
    # output layer
    fc1 = tf.contrib.layers.flatten(pool2)
    out_layer = tf.layers.dense(fc1, num_classes)
    
    return out_layer

In [ ]:
# Construct model
prediction = neural_net_cnn(X)

# Define loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=Y))
optimizer = tf.train.AdamOptimizer()
train = optimizer.minimize(loss)

# Evaluate model (uses prediction accuracy)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing tensorboard and summary object <----[Tensorboard] Define which metric to record
tf.summary.scalar('accuracy', accuracy)
tf.summary.scalar('loss', loss)
merged_summary = tf.summary.merge_all()


In [ ]:
saver = tf.train.Saver([x for x in tf.global_variables()]) # <----[save/load] initialize saver object
from_scratch=False

# Start training
with tf.Session() as sess:

    if from_scratch==True:
        sess.run(tf.global_variables_initializer())
    else:
        saver.restore(sess, './saved_model/cnn_model-200') # <----[save/load] loading previous trained model
    
    train_writer = tf.summary.FileWriter('./train_progress', sess.graph) # <----[Tensorboard] record the graph
    
    for step in range(num_steps):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        training_progress, _ = sess.run([merged_summary,train], feed_dict={X: batch_x, Y: batch_y})
        train_writer.add_summary(training_progress, step) # <----[Tensorboard] record training progress
        if step%100==0:
            saver.save(sess, './saved_model/cnn_model', global_step=step) # <----[save/load] saving model

    print("Optimization Finished!")
    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: mnist.test.images,
                                      Y: mnist.test.labels}))